In [1]:
%run ./setting.ipynb

In [13]:
from sklearn import preprocessing

# join feature and label

In [2]:
pd.read_parquet(
    os.path.join(
    data_path,
    "NDX_features.parquet")
).to_sql(
    "features", 
    conn,
    if_exists = "replace",
    )

9514

In [3]:
pd.read_parquet(
    os.path.join(
    data_path,
    "NDX_daily_label.parquet")
).to_sql(
    "labels", 
    conn,
    if_exists = "replace",
    )

9514

In [4]:
features = sql(f"select * from features").columns

sql_features = []
for c in features:
    if 'feature_' in c:
        sql_features.append(f"b.{c}")

sql_features = ",\n".join(sql_features)

In [5]:
data = sql(f"""
select 
a.*,
{sql_features}
from labels as a
left join features as b
on a.Date = b.Date
""")

# features

In [6]:
feature_name = []
categorical_feature = []
numemrical_feature = []

for c in data.columns:
    if 'feature_' in c:
        feature_name.append(c)
    if 'feature_cat_' in c:
        categorical_feature.append(c)
    if 'feature_num_' in c:
        numemrical_feature.append(c)

In [7]:
np.save(
    os.path.join(
    data_path,
    'feature_name.npy',
    ),
    np.array(feature_name),    
    )

In [8]:
np.save(
    os.path.join(
    data_path,
    'categorical_feature.npy',
    ),
    np.array(categorical_feature),    
    )

In [9]:
np.save(
    os.path.join(
    data_path,
    'numemrical_feature.npy',
    ),
    np.array(numemrical_feature),    
    )

## filling the missing values

In [10]:
%%time

X_filled = data.copy()

for c in feature_name:
    if c not in categorical_feature:
        X_filled[c] = X_filled[c].fillna(X_filled[c].median())
        X_filled[c] = X_filled[c].replace([np.inf, -np.inf], 1000000,)
    if c in categorical_feature:
        X_filled[c] = X_filled[c].fillna('')

CPU times: total: 31.2 ms
Wall time: 43.5 ms


In [11]:
X_filled.to_parquet(
    os.path.join(
    data_path,
    'data_filled.parquet',
    ),
    index = False,
    )

# encodding the categorical features

In [14]:
%%time

X_encoded = data.copy()

for c in categorical_feature:   
    le = preprocessing.LabelEncoder()
    le.fit(X_encoded[c])
    X_encoded[c] = le.transform(X_encoded[c])

CPU times: total: 15.6 ms
Wall time: 19 ms


In [15]:
X_encoded.to_parquet(
    os.path.join(
    data_path,
    'data_encoded.parquet',
    ),
    index = False,
    )

# labes

In [12]:
y = data['label__close_to_open_ratio']

# end